In [6]:
import numpy as np
import os 
import torch 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as albu
import torch.nn as nn
import pandas as pd
!pip install -q segmentation-models-pytorch
import segmentation_models_pytorch as smp

In [7]:
IMAGE_PATH= '../input/hubmap-256x256/train'
MASK_PATH= '../input/hubmap-256x256/masks'

In [8]:
def create_df(path):
    name = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            name.append(filename.split('.')[0])
    
    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

df = create_df(IMAGE_PATH)
print('Total Images: ', len(df))

Total Images:  9580


In [9]:
df.head()

,id
0,8242609fa_0777
1,e79de561c_0309
2,2f6ecfcdf_0197
3,afa5e8098_1083
4,aaa6a05cc_0033


In [68]:
df= df.iloc[:80,]

In [11]:
from sklearn.model_selection import train_test_split

In [69]:
X_train, X_val = train_test_split(df['id'].values, test_size=0.15)

In [13]:
mean = [0.65459856,0.48386562,0.69428385]
std = [0.15167958,0.23584107,0.13146145]

In [14]:
from torchvision import transforms as T
from PIL import Image

class hubmap(Dataset):
    def __init__(self, img_dir, mask_dir, classes=1,augment=None):
        self.img_dir=img_dir
        self.mask_dir=mask_dir
        self.ids=os.listdir(img_dir)
        self.img_path=[os.path.join(img_dir,i) for i in self.ids]
        self.mask_id= os.listdir(mask_dir)
        self.mask_fps= [os.path.join(mask_dir, i) for i in self.mask_id]
        self.classes=classes
        self.augment=augment
        
    def __len__(self):
        len(self.img_path)
        
    def __getitem__(self,indx):
        image=cv2.imread(self.img_path[indx])
        mask=cv2.imread(self.mask_fps[indx]) 
        
        if self.augment is not None:
            sample = self.augment(image=image, mask=mask)
            img, mask = sample['image'], sample['mask']
            
        return img2tensor((img/255.0 - mean)/std),img2tensor(mask)

def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

In [55]:
class hubmap(Dataset):
    
    def __init__(self, img_path, mask_path, X, mean, std, transform=None):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.mean = mean
        self.std = std
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.img_path +'/'+ self.X[idx] + '.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path +'/'+ self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)
        
        
        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = Image.fromarray(aug['image'])
            mask = aug['mask']
        
        if self.transform is None:
            img = Image.fromarray(img)
        
        t = T.Compose([T.ToTensor(), T.Normalize(self.mean, self.std)])
        img = t(img)        
        mask = torch.from_numpy(mask)
        mask=torch.unsqueeze(mask,0)
        return img,mask

In [16]:
train_transforms= albu.Compose([
            albu.HorizontalFlip(p=0.5),
            albu.VerticalFlip(p=0.5),
            albu.ShiftScaleRotate(rotate_limit=25, scale_limit=0.15, shift_limit=0, p=0.75),
            albu.CoarseDropout(max_holes=16, max_height=64 ,max_width=64 ,p=0.5),
            albu.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.25, p=0.75),
            albu.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, p=0.5),
    
])


In [70]:
train_set = hubmap(IMAGE_PATH, MASK_PATH, X_train, mean, std, train_transforms)
val_set = hubmap(IMAGE_PATH, MASK_PATH, X_val, mean, std, None )

In [71]:
train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
val_loader = DataLoader(val_set, batch_size=3, shuffle=True)     

In [70]:
import torch.nn as nn

In [19]:
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' 

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=1, 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [20]:
loss_fn = nn.BCEWithLogitsLoss ()
optimizer= torch.optim.Adam(model.parameters(), lr=1e-3)
lr=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

https://towardsdatascience.com/semantic-hand-segmentation-using-pytorch-3e7a0a0386fa#:~:text=Semantic%20hand%20segmentation%20using%20Pytorch.%20Semantic%20segmentation%20is,differentiate%20between%20multiple%20instances%20of%20the%20same%20class.

In [21]:
def meanIOU(target, predicted):
    if target.shape != predicted.shape:
        print("target has dimension", target.shape, ", predicted values have shape", predicted.shape)
        return
        
    if target.dim() != 4:
        print("target has dim", target.dim(), ", Must be 4.")
        return
    
    iousum = 0
    for i in range(target.shape[0]):
        target_arr = target[i, :, :, :].clone().detach().cpu().numpy().argmax(0)
        predicted_arr = predicted[i, :, :, :].clone().detach().cpu().numpy().argmax(0)
        
        intersection = np.logical_and(target_arr, predicted_arr).sum()
        union = np.logical_or(target_arr, predicted_arr).sum()
        if union == 0:
            iou_score = 0
        else :
            iou_score = intersection / union
        iousum +=iou_score
        
    miou = iousum/target.shape[0]
    return miou

In [65]:
def training_loop(n_epochs, optimizer,lr_scheduler,model,loss_fn, train_loader, val_loader):
    tr_loss_arr = []
    val_loss_arr = []
    meanioutrain = []
    meanioutest = []
        
    for epoch in range(0, n_epochs):
        train_loss = 0.0
        meaniou = 0
        
        for X, y in train_loader:
            model.train()
            ypred = model(X)
            loss = loss_fn(ypred, y.float())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            tr_loss_arr.append(loss.item())
            meanioutrain.append(meanIOU(y, ypred))
            
        with torch.no_grad():
            
            val_loss = 0
            for X, y in val_loader:
                model.eval()
                ypred = model(X)
                
                val_loss_arr.append(loss_fn(ypred, y.float()).item())
                meanioutest.append(meanIOU(y, ypred))
                
        lr_scheduler.step(loss)

    return tr_loss_arr, val_loss_arr, meanioutrain, meanioutest

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [72]:
history= training_loop(10,optimizer,lr,model,loss_fn,train_loader, val_loader)

Epoch 00013: reducing learning rate of group 0 to 1.0000e-05.
Epoch 00019: reducing learning rate of group 0 to 1.0000e-06.


In [67]:
if torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')

using device: cuda


In [75]:
import matplotlib.pyplot as plt

In [88]:
train_set[0][0].shape

torch.Size([3, 256, 256])

In [92]:
torch.unsqueeze(train_set[0][0],0).shape

torch.Size([1, 3, 256, 256])

In [99]:
torch.squeeze(ypred).shape

torch.Size([256, 256])